# Intent 와 NER 모델을 만들기 위한 Data의 구성 방법

In [204]:
train_data_order = ['판교에 지금 피자 주문해줘']
train_data_reserve = ['오늘 날짜에 호텔 예약 해줄레']
train_data_info = ['오늘 날짜에 판교 여행 정보 알려줘']

dict_entity = {
    'date' : ['오늘','내일','모래'],
    'loc' : ['판교','야탑','서현'],
    'menu' : ['피자','햄버거','치킨'],
    'hotel' : ['호텔','여관','민박'],
    'travel' : ['여행','놀러갈','구경']
}

length = 1
for key in list(dict_entity.keys()):
    length = length * len(dict_entity[key])
print(length)

243


# Vector 구성 (a~z / 한글)

In [208]:
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
vocab = (" $%'()+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ"
           "\\^_abcdefghijklmnopqrstuvwxyz{|}")

# 형태소 분석

In [209]:
from konlpy.tag import Mecab
mecab = Mecab('/usr/local/lib/mecab/dic/mecab-ko-dic')
morpphed_text = mecab.pos(train_data_order[0])

[('판교', 'NNG'),
 ('에', 'JKB'),
 ('지금', 'MAG'),
 ('피자', 'NNG'),
 ('주문', 'NNG'),
 ('해', 'XSV+EC'),
 ('줘', 'VX+EC')]

# Intent 학습 Data의 구성 

- Feature Engineering으로 Intent의 정확도를 높임)

In [ ]:

pattern = "tagloc 에 tagdate tagmenu 주문 해줘"

# Data augmentation (Entity and Pattern)

In [203]:
def augmentation_pattern(pattern, dict_entity):
    #입력된 패턴을 List로 바꿈
    aug_pattern = pattern.split(' ')
    #Augment된 Text List
    augmented_text_list = []
    #copy를 위한 임시 List
    temp_aug = []
    for i in range(0,len(aug_pattern)):
        #Entity에 해당하는 값일 경우 Entity List를 가져옴
        if(aug_pattern[i].find("tag") > -1):
            dict_list = dict_entity[aug_pattern[i].replace("tag","")]
            #각 Entity별로 값을 append하면서 Pattern구성
            for j in range(0,len(dict_list)):
                #최초 Entity값은 그냥 추가만함
                if(i == 0):
                    augmented_text_list.append(dict_list[j] + " ")
                elif(j == 1):
                    augmented_text_list = list(filter(lambda word:len(word.split(' ')) == i+1 ,augmented_text_list))
                    copy_data_order = augmented_text_list * (len(dict_list)-2)
                    augmented_text_list = list(map(lambda x:x + dict_list[j] + " ",augmented_text_list))
                    augmented_text_list = augmented_text_list + temp_aug + copy_data_order
                else:
                    #List의 수를 체크하여 값을 추가
                    temp_aug = list(filter(lambda word:len(word.split(' ')) == i+1 ,augmented_text_list))
                    temp_aug = list(map(lambda x:x + dict_list[j] + " " ,temp_aug))
                    #추가된 List를 위해 기존 값 삭제
                    if(j != 0):
                        augmented_text_list = augmented_text_list[0:len(augmented_text_list) - len(temp_aug)]
                    augmented_text_list = augmented_text_list + temp_aug
        #Entity추가 대상이 아닐 경우 Pattern만 추가
        else:
            augmented_text_list = list(map(lambda x:x + aug_pattern[i] + " ",augmented_text_list))
        #N*N으로 증가시키기 위한 List
        temp_aug = augmented_text_list

    print(augmented_text_list)
    print(len(augmented_text_list))

augmentation_pattern(pattern, dict_entity)

['판교 에 내일 햄버거 주문 해줘 ', '야탑 에 내일 햄버거 주문 해줘 ', '서현 에 내일 햄버거 주문 해줘 ', '판교 에 오늘 햄버거 주문 해줘 ', '야탑 에 오늘 햄버거 주문 해줘 ', '서현 에 오늘 햄버거 주문 해줘 ', '판교 에 모래 햄버거 주문 해줘 ', '야탑 에 모래 햄버거 주문 해줘 ', '서현 에 모래 햄버거 주문 해줘 ', '판교 에 내일 피자 주문 해줘 ', '야탑 에 내일 피자 주문 해줘 ', '서현 에 내일 피자 주문 해줘 ', '판교 에 오늘 피자 주문 해줘 ', '야탑 에 오늘 피자 주문 해줘 ', '서현 에 오늘 피자 주문 해줘 ', '판교 에 모래 피자 주문 해줘 ', '야탑 에 모래 피자 주문 해줘 ', '서현 에 모래 피자 주문 해줘 ', '판교 에 내일 치킨 주문 해줘 ', '야탑 에 내일 치킨 주문 해줘 ', '서현 에 내일 치킨 주문 해줘 ', '판교 에 오늘 치킨 주문 해줘 ', '야탑 에 오늘 치킨 주문 해줘 ', '서현 에 오늘 치킨 주문 해줘 ', '판교 에 모래 치킨 주문 해줘 ', '야탑 에 모래 치킨 주문 해줘 ', '서현 에 모래 치킨 주문 해줘 ']
27
